En el siguiente ejercicio se presenta un set de datos con el que trataremos de clasificar si un cliente es solvente o no a la hora de devolver un crédito solicitado a una entidad bancaria.

Para ponernos en situación pertenecemos al equipo de Data Science de una entidad bancaria y se no pide realizar un modelo que sea capaz de evaluar el riesgo de un cliente cuando nos solicita un crédito a través de una tarjeta de crédito en base al histórico de datos que os pasan desde el equipo de negocio (datos extraidos de kaggle).

A continuación os dejan una breve explicación de en que consiste cada una de las variables del conjunto de datos:

- BAD: 1 = candidato con préstamo incumplido o con mora; 0 = candidato que paga su deuda y no tiene registro negativo
- LOAN: Cantidad de solicitud de préstamo
- MORTDUE: Cantidad adeudada de la hipoteca existente
- VALUE: Valor actual del bien o propiedad
- REASON: DebtCon = consolidación de la deuda; HomeImp = mejoras para el hogar
- JOB: Categorias ocupacionales o profesionales
- YOJ: Años es esu trabajo actual
- DEROG: Número de informes derogados o cancelados importantes
- DELINQ: Número de lineas de crédito morosas
- CLAGE: Antiguedad de la linea de crédito más antigua en meses
- NINQ:Número de consultas crediticas recientes
- CLNO: Número de líneas de crédito

En este caso la target de nuestro modelo será **BAD**, el resto las consideraremos variables predictoras.

Que se espera que hagais:

- Análisis descriptivo de los datos (esto ayudará a realizar una mejor elección del modelo a usar)
- Tratamiento y limpieza de los datos
- Prueba y evaluación de diferentes modelos
- Breve explicación de los resultados, justificando la elección final del modelo (no me vale con que es el que mejor métrica tenía)
- Que disfruteis del ejercico :).

In [1]:
import LibreriasClasificadores

In [2]:

df1 = pd.read_csv(r'D:\bootcamp\dsb06rt\mod5-machine-learning-y-deep-learning\29_11_2023\credit_risk.csv')
df1.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN


TRATAMIENTO DE LOS DATOS PARA VER QUE METODO TIENE MAS PRECISIO SIN NAN

In [3]:
df_no_nan=df1.dropna().copy()

In [4]:
df_no_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3364 entries, 5 to 5959
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAD      3364 non-null   int64  
 1   LOAN     3364 non-null   int64  
 2   MORTDUE  3364 non-null   float64
 3   VALUE    3364 non-null   float64
 4   REASON   3364 non-null   object 
 5   JOB      3364 non-null   object 
 6   YOJ      3364 non-null   float64
 7   DEROG    3364 non-null   float64
 8   DELINQ   3364 non-null   float64
 9   CLAGE    3364 non-null   float64
 10  NINQ     3364 non-null   float64
 11  CLNO     3364 non-null   float64
 12  DEBTINC  3364 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 367.9+ KB


Tratamos las categoricas

In [5]:
df_no_nan.REASON.value_counts()

DebtCon    2369
HomeImp     995
Name: REASON, dtype: int64

In [6]:
df_no_nan.JOB.value_counts()

Other      1286
ProfExe     899
Office      577
Mgr         450
Self         99
Sales        53
Name: JOB, dtype: int64

In [7]:
dic_re={'DebtCon':1,'HomeImp':2}
dic_job={'Other':1,'ProfExe':2,'Office':3,'Mgr':4,'Self':5,'Sales':6}

In [8]:
df_no_nan['JobEnum']=df_no_nan['JOB'].map(dic_job)
df_no_nan['ReasonEnum']=df_no_nan['REASON'].map(dic_re)

In [9]:
df_no_nan.head(3)

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,JobEnum,ReasonEnum
5,1,1700,30548.0,40320.0,HomeImp,Other,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614,1,2
7,1,1800,28502.0,43034.0,HomeImp,Other,11.0,0.0,0.0,88.766030,0.0,8.0,36.884894,1,2
19,0,2300,102370.0,120953.0,HomeImp,Office,2.0,0.0,0.0,90.992533,0.0,13.0,31.588503,3,2


Funcion para realizar modelos con diferentes configuraciones de momento todos los modelos se llamaran con sus ajustes por defecto

In [3]:
#X e y ya deben estar tratadas si tienes que aplicar a y train por separado alguna modificacion 
#tienes que crear otra funcion 
#1 BOOLEANO es para saber si quieres normalizar en base a X_train
#2 Booleano es para saber si quieres  hacer xtrain stratify=y
def VecinosKneighbors(X,y,normalizar,stratify):
    if(normalizar)and(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = KNeighborsClassifier(n_neighbors = 3)
        model.fit(X_train, y_train)
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
#         confusion_matrix(y_test, yhat, labels = [0, 1])
     
#         plt.figure(figsize=(10,10))
#         sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         plt.xlabel('Predicción')
#         plt.ylabel('Verdad');
#         plt.title(f'MODELO NORMALIZADO CON STRATIFY EN Y')
    elif(normalizar)and not(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = KNeighborsClassifier(n_neighbors = 3)
        model.fit(X_train, y_train)
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
    elif(stratify)and not(normalizar):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        model = KNeighborsClassifier(n_neighbors = 3)
        model.fit(X_train, y_train)
        yhat = model.predict(X_test)
        print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
    else:
         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
         model = KNeighborsClassifier(n_neighbors = 3)
         model.fit(X_train, y_train)
         yhat = model.predict(X_test)
         print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
         print("Exactitud:"    , accuracy_score(y_test, yhat))
         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
         print("ROC AUC:", roc_auc_score(y_test, yhat))
#          confusion_matrix(y_test, yhat, labels = [0, 1])
#          plt.figure(figsize=(10,10))
#          sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#          plt.xlabel('Predicción')
#          plt.ylabel('Verdad');
#          plt.title(f'MODELO SIN NORMALIZADO Y SIN STRATIFY EN Y')


#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------

    
#COMENTO LA FUNCION ENTERA HASTA ENTENDERLA MEJOR    
# def VecinosRadiusNeighbors(X,y,normalizar,stratify):
#     if(normalizar)and(stratify):
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
#         scaler = MinMaxScaler()
#         scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
#         X_train_scaler = scaler.transform(X_train)
#         X_test_scaler = scaler.transform(X_test)
#         model = RadiusNeighborsClassifier(radius = 0.5)
#         model.fit(X_train_scaler, y_train)
#         yhat = model.predict(X_test)
#         print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
#         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
#         print("Exactitud:"    , accuracy_score(y_test, yhat))
#         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
#         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
#         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
#         print("ROC AUC:", roc_auc_score(y_test, yhat))
# #         confusion_matrix(y_test, yhat, labels = [0, 1])
     
# #         plt.figure(figsize=(10,10))
# #         sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
# #         plt.xlabel('Predicción')
# #         plt.ylabel('Verdad');
# #         plt.title(f'MODELO NORMALIZADO CON STRATIFY EN Y')
#     elif(normalizar)and not(stratify):
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
#         scaler = MinMaxScaler()
#         scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
#         X_train_scaler = scaler.transform(X_train)
#         X_test_scaler = scaler.transform(X_test)
#         model = RadiusNeighborsClassifier(radius = 0.5)
#         model.fit(X_train_scaler, y_train)
#         yhat = model.predict(X_test)
#         print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
#         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
#         print("Exactitud:"    , accuracy_score(y_test, yhat))
#         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
#         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
#         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
#         # confusion_matrix(y_test, yhat, labels = [0, 1])
#         # plt.figure(figsize=(10,10))
#         # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         # plt.xlabel('Predicción')
#         # plt.ylabel('Verdad');
#         # plt.title(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
#     elif(stratify)and not(normalizar):
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
#         model = RadiusNeighborsClassifier(radius = 0.5)
#         model.fit(X_train_scaler, y_train)
#         yhat = model.predict(X_test)
#         print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
#         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
#         print("Exactitud:"    , accuracy_score(y_test, yhat))
#         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
#         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
#         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
#         print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
#         # confusion_matrix(y_test, yhat, labels = [0, 1])
#         # plt.figure(figsize=(10,10))
#         # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         # plt.xlabel('Predicción')
#         # plt.ylabel('Verdad');
#         # plt.title(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
#     else:
#          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
#          model = RadiusNeighborsClassifier(radius = 0.5)
#          model.fit(X_train_scaler, y_train)
#          yhat = model.predict(X_test)
#          print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
#          print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
#          print("Exactitud:"    , accuracy_score(y_test, yhat))
#          print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
#          print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
#          print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
         
# #          confusion_matrix(y_test, yhat, labels = [0, 1])
# #          plt.figure(figsize=(10,10))
# #          sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
# #          plt.xlabel('Predicción')
# #          plt.ylabel('Verdad');
# #          plt.title(f'MODELO SIN NORMALIZADO Y SIN STRATIFY EN Y')


#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------


def LogisticRegresion(X,y,normalizar,stratify):
    if(normalizar)and(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = LogisticRegression()
        model.fit(X_train, y_train)
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
#         confusion_matrix(y_test, yhat, labels = [0, 1])
     
#         plt.figure(figsize=(10,10))
#         sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         plt.xlabel('Predicción')
#         plt.ylabel('Verdad');
#         plt.title(f'MODELO NORMALIZADO CON STRATIFY EN Y')
    elif(normalizar)and not(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = LogisticRegression()
        model.fit(X_train, y_train)
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
    elif(stratify)and not(normalizar):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        model = LogisticRegression()
        model.fit(X_train, y_train)
        yhat = model.predict(X_test)
        print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
    else:
         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
         model = LogisticRegression()
         model.fit(X_train, y_train)
         yhat = model.predict(X_test)
         print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
         print("Exactitud:"    , accuracy_score(y_test, yhat))
         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
         print("ROC AUC:", roc_auc_score(y_test, yhat))
        
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------

    
def nearestCentroid(X,y,normalizar,stratify):
    if(normalizar)and(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = NearestCentroid()
        model.fit(X_train, y_train)  
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
#         confusion_matrix(y_test, yhat, labels = [0, 1])
     
#         plt.figure(figsize=(10,10))
#         sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         plt.xlabel('Predicción')
#         plt.ylabel('Verdad');
#         plt.title(f'MODELO NORMALIZADO CON STRATIFY EN Y')
    elif(normalizar)and not(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = NearestCentroid()
        model.fit(X_train, y_train)  
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
    elif(stratify)and not(normalizar):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        model = NearestCentroid()
        model.fit(X_train, y_train)  
        yhat = model.predict(X_test)
        print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
 
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
    else:
         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
         model = NearestCentroid()
         model.fit(X_train, y_train)  
         yhat = model.predict(X_test)
         print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
         print("Exactitud:"    , accuracy_score(y_test, yhat))
         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
         print("ROC AUC:", roc_auc_score(y_test, yhat))
    

#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
    
    
def ArbolDecision(X,y,normalizar,stratify,min_samples):
    if(normalizar)and(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = DecisionTreeClassifier(min_samples_leaf=min_samples)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
#         confusion_matrix(y_test, yhat, labels = [0, 1])
     
#         plt.figure(figsize=(10,10))
#         sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         plt.xlabel('Predicción')
#         plt.ylabel('Verdad');
#         plt.title(f'MODELO NORMALIZADO CON STRATIFY EN Y')
    elif(normalizar)and not(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = DecisionTreeClassifier(min_samples_leaf=min_samples)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
    elif(stratify)and not(normalizar):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        model = DecisionTreeClassifier(min_samples_leaf=min_samples)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
    else:
         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
         model = DecisionTreeClassifier(min_samples_leaf=min_samples)
         model.fit(X_train, y_train) 
         yhat = model.predict(X_test)
         print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
         print("Exactitud:"    , accuracy_score(y_test, yhat))
         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
         print("ROC AUC:", roc_auc_score(y_test, yhat))    

#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------

    
    
def ArbolRandom(X,y,normalizar,stratify):
    if(normalizar)and(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = RandomForestClassifier()
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
#         confusion_matrix(y_test, yhat, labels = [0, 1])
     
#         plt.figure(figsize=(10,10))
#         sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         plt.xlabel('Predicción')
#         plt.ylabel('Verdad');
#         plt.title(f'MODELO NORMALIZADO CON STRATIFY EN Y')
    elif(normalizar)and not(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = RandomForestClassifier()
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
    elif(stratify)and not(normalizar):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        model = RandomForestClassifier()
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
    else:
         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
         model = RandomForestClassifier()
         model.fit(X_train, y_train) 
         yhat = model.predict(X_test)
         print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
         print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
         print("Exactitud:"    , accuracy_score(y_test, yhat))
         print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
         print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
         print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
         print("ROC AUC:", roc_auc_score(y_test, yhat))        
    
    
    
    
    
    
    
    
    
    
    
    

Comparativa de formas

In [47]:
X=df_no_nan.drop(['BAD','JOB','REASON'],axis=1)
y=df_no_nan['BAD']

In [48]:
print('------------------------------------------------------------------')
VecinosKneighbors(X,y,1,1)
print('------------------------------------------------------------------')
VecinosKneighbors(X,y,1,0)
print('------------------------------------------------------------------')
VecinosKneighbors(X,y,0,1)
print('------------------------------------------------------------------')
VecinosKneighbors(X,y,0,0)
print('------------------------------------------------------------------')

------------------------------------------------------------------
MODELO NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.5153405902816743
Exactitud: 0.912332838038633
Precisión: 0.7271529888551165
Sensibilidad: 0.5609570418705818
F1-score: 0.5834565958562812
ROC AUC: 0.5609570418705818
------------------------------------------------------------------
MODELO NORMALIZADO SIN STRATIFY EN Y
Jaccard Index: 0.5326710066070734
Exactitud: 0.9212481426448736
Precisión: 0.7642857142857142
Sensibilidad: 0.5754949062282936
F1-score: 0.6059762496547915
ROC AUC: 0.5754949062282937
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.5153405902816743
Exactitud: 0.912332838038633
Precisión: 0.7271529888551165
Sensibilidad: 0.5609570418705818
F1-score: 0.5834565958562812
ROC AUC: 0.5609570418705818
MODELO SIN NORMALIZADO CON STRATIFY EN Y
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON

In [ ]:
# print('------------------------------------------------------------------')
# VecinosRadiusNeighbors(X,y,1,1)
# print('------------------------------------------------------------------')
# VecinosRadiusNeighbors(X,y,1,0)
# print('------------------------------------------------------------------')
# VecinosRadiusNeighbors(X,y,0,1)
# print('------------------------------------------------------------------')
# VecinosRadiusNeighbors(X,y,0,0)
# print('------------------------------------------------------------------')

In [49]:
X=df_no_nan.drop(['BAD','JOB','REASON'],axis=1)
y=df_no_nan['BAD']

In [50]:
print('------------------------------------------------------------------')
LogisticRegresion(X,y,1,1)
print('------------------------------------------------------------------')
LogisticRegresion(X,y,1,0)
print('------------------------------------------------------------------')
LogisticRegresion(X,y,0,1)
print('------------------------------------------------------------------')
LogisticRegresion(X,y,0,0)
print('------------------------------------------------------------------')

------------------------------------------------------------------
MODELO NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.4554234769687964
Exactitud: 0.9108469539375929
Precisión: 0.4554234769687964
Sensibilidad: 0.5
F1-score: 0.4766718506998445
ROC AUC: 0.5
------------------------------------------------------------------
MODELO NORMALIZADO SIN STRATIFY EN Y
Jaccard Index: 0.45839524517087665
Exactitud: 0.9167904903417533
Precisión: 0.45839524517087665
Sensibilidad: 0.5
F1-score: 0.4782945736434109
ROC AUC: 0.5
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.4554234769687964
Exactitud: 0.9108469539375929
Precisión: 0.4554234769687964
Sensibilidad: 0.5
F1-score: 0.4766718506998445
MODELO SIN NORMALIZADO CON STRATIFY EN Y
ROC AUC: 0.5
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.45839524517087665
Exactitud: 0.9167904903417533
Precis

C:\Users\Danie\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danie\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danie\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danie\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedM

In [51]:
X=df_no_nan.drop(['BAD','JOB','REASON'],axis=1)
y=df_no_nan['BAD']

In [52]:
print('------------------------------------------------------------------')
nearestCentroid(X,y,1,1)
print('------------------------------------------------------------------')
nearestCentroid(X,y,1,0)
print('------------------------------------------------------------------')
nearestCentroid(X,y,0,1)
print('------------------------------------------------------------------')
nearestCentroid(X,y,0,0)
print('------------------------------------------------------------------')

------------------------------------------------------------------
MODELO NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.22487618911889407
Exactitud: 0.399702823179792
Precisión: 0.50175983436853
Sensibilidad: 0.5050842849374659
F1-score: 0.3459103156274057
ROC AUC: 0.5050842849374659
------------------------------------------------------------------
MODELO NORMALIZADO SIN STRATIFY EN Y
Jaccard Index: 0.22045834874667514
Exactitud: 0.3922734026745914
Precisión: 0.5074037983874049
Sensibilidad: 0.5224299606390368
F1-score: 0.34083599746159465
ROC AUC: 0.5224299606390368
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.22487618911889407
Exactitud: 0.399702823179792
Precisión: 0.50175983436853
Sensibilidad: 0.5050842849374659
F1-score: 0.3459103156274057
ROC AUC: 0.5050842849374659
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.220458348

In [53]:
X=df_no_nan.drop(['BAD','JOB','REASON'],axis=1)
y=df_no_nan['BAD']

In [54]:
print('------------------------------------------------------------------')
ArbolDecision(X,y,1,1,4)
print('------------------------------------------------------------------')
ArbolDecision(X,y,1,0,4)
print('------------------------------------------------------------------')
ArbolDecision(X,y,0,1,4)
print('------------------------------------------------------------------')
ArbolDecision(X,y,0,0,4)
print('------------------------------------------------------------------')

------------------------------------------------------------------
MODELO NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.7088131993792371
Exactitud: 0.9405646359583952
Precisión: 0.8240142404116556
Sensibilidad: 0.7944671016856988
F1-score: 0.8083276372750057
ROC AUC: 0.7944671016856988
------------------------------------------------------------------
MODELO NORMALIZADO SIN STRATIFY EN Y
Jaccard Index: 0.6428482972136222
Exactitud: 0.9286775631500743
Precisión: 0.7703522640593579
Sensibilidad: 0.7256743459134058
F1-score: 0.7454132778513335
ROC AUC: 0.7256743459134058
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.7088131993792371
Exactitud: 0.9405646359583952
Precisión: 0.8240142404116556
Sensibilidad: 0.7944671016856988
F1-score: 0.8083276372750057
ROC AUC: 0.7944671016856988
------------------------------------------------------------------
MODELO SIN NORMALIZADO SIN STRATIFY EN Y
Jaccard Index: 0.6568971

In [55]:
X=df_no_nan.drop(['BAD','JOB','REASON'],axis=1)
y=df_no_nan['BAD']

In [56]:
print('------------------------------------------------------------------')
ArbolRandom(X,y,1,1)
print('------------------------------------------------------------------')
ArbolRandom(X,y,1,0)
print('------------------------------------------------------------------')
ArbolRandom(X,y,0,1)
print('------------------------------------------------------------------')
ArbolRandom(X,y,0,0)
print('------------------------------------------------------------------')

------------------------------------------------------------------
MODELO NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.7448257930317213
Exactitud: 0.9583952451708767
Precisión: 0.9781591263650546
Sensibilidad: 0.7666666666666666
F1-score: 0.8366618126343527
ROC AUC: 0.7666666666666666
------------------------------------------------------------------
MODELO NORMALIZADO SIN STRATIFY EN Y
Jaccard Index: 0.7379658385093169
Exactitud: 0.9598811292719168
Precisión: 0.9790372670807453
Sensibilidad: 0.7589285714285714
F1-score: 0.8304706815319308
ROC AUC: 0.7589285714285714
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.7629890453834116
Exactitud: 0.9613670133729569
Precisión: 0.9796557120500782
Sensibilidad: 0.7833333333333333
F1-score: 0.8513187410781048
ROC AUC: 0.7833333333333333
------------------------------------------------------------------
MODELO SIN NORMALIZADO CON STRATIFY EN Y
Jaccard Index: 0.7573153

De momento el ganador es el RamdonForest pero vamos a hacer un bucle para foresDecision

In [45]:
a=ArbolDecision2(X,y,1,1,1)
info_leaf=1
b=0
for i in range(1,1000):
    

    b=ArbolDecision2(X,y,1,1,i)
    if b>a:
        a=b
        info_leaf=i
    else:
        pass

    b=ArbolDecision2(X,y,1,0,i)
    if b>a:
        a=b
        info_leaf=i
    else:
        pass

    b=ArbolDecision2(X,y,0,1,i)
    if b>a:
        a=b
        info_leaf=i
    else:
        pass

    b=ArbolDecision2(X,y,0,0,i)
    if b>a:
        a=b
        info_leaf=i
    else:
        pass
    
    
    

In [46]:
print(f'El numero de leaf es: {info_leaf} el mayor f1 es: {a}')

El numero de leaf es: 7 el mayor f1 es: 0.8426300981346126


´funcion para el bucle

In [43]:

def ArbolDecision2(X, y, normalizar, stratify, min_samples):
    if normalizar and stratify:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = DecisionTreeClassifier(min_samples_leaf=min_samples)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        # print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        # print("Jaccard Index:", jaccard_score(y_test, yhat, average="macro"))
        # print("Exactitud:", accuracy_score(y_test, yhat))
        # print("Precisión:", precision_score(y_test, yhat, average="macro"))
        # print("Sensibilidad:", recall_score(y_test, yhat, average="macro"))
        # print("F1-score:", f1_score(y_test, yhat, average="macro"))
        # print("ROC AUC:", roc_auc_score(y_test, yhat))
        return f1_score(y_test, yhat, average="macro")

    elif normalizar and not stratify:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = DecisionTreeClassifier(min_samples_leaf=min_samples)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        # print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        # print("Jaccard Index:", jaccard_score(y_test, yhat, average="macro"))
        # print("Exactitud:", accuracy_score(y_test, yhat))
        # print("Precisión:", precision_score(y_test, yhat, average="macro"))
        # print("Sensibilidad:", recall_score(y_test, yhat, average="macro"))
        # print("F1-score:", f1_score(y_test, yhat, average="macro"))
        # print("ROC AUC:", roc_auc_score(y_test, yhat))
        return f1_score(y_test, yhat, average="macro")

    elif stratify and not normalizar:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
        model = DecisionTreeClassifier(min_samples_leaf=min_samples)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        # print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        # print("Jaccard Index:", jaccard_score(y_test, yhat, average="macro"))
        # print("Exactitud:", accuracy_score(y_test, yhat))
        # print("Precisión:", precision_score(y_test, yhat, average="macro"))
        # print("Sensibilidad:", recall_score(y_test, yhat, average="macro"))
        # print("F1-score:", f1_score(y_test, yhat, average="macro"))
        # print("ROC AUC:", roc_auc_score(y_test, yhat))
        return f1_score(y_test, yhat, average="macro")

    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
        model = DecisionTreeClassifier(min_samples_leaf=min_samples)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        # print(f'MODELO SIN NORMALIZADO SIN STRATIFY EN Y')
        # print("Jaccard Index:", jaccard_score(y_test, yhat, average="macro"))
        # print("Exactitud:", accuracy_score(y_test, yhat))
        # print("Precisión:", precision_score(y_test, yhat, average="macro"))
        # print("Sensibilidad:", recall_score(y_test, yhat, average="macro"))
        # print("F1-score:", f1_score(y_test, yhat, average="macro"))
        # print("ROC AUC:", roc_auc_score(y_test, yhat))
        return f1_score(y_test, yhat, average="macro")

In [4]:
def ArbolRandom2(X,y,normalizar,stratify,i):
    if(normalizar)and(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = RandomForestClassifier(max_depth=i)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        print(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        print("Exactitud:"    , accuracy_score(y_test, yhat))
        print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        print("ROC AUC:", roc_auc_score(y_test, yhat))
#         confusion_matrix(y_test, yhat, labels = [0, 1])
     
#         plt.figure(figsize=(10,10))
#         sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
#         plt.xlabel('Predicción')
#         plt.ylabel('Verdad');
#         plt.title(f'MODELO NORMALIZADO CON STRATIFY EN Y')
        return f1_score(y_test, yhat, average="macro")
    elif(normalizar)and not(stratify):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
        scaler = MinMaxScaler()
        scaler.fit(X_train)#aplicamos el scaler a xtrain y con el scaler resultante transformamos xtrain y xtest
        X_train_scaler = scaler.transform(X_train)
        X_test_scaler = scaler.transform(X_test)
        model = RandomForestClassifier(max_depth=i)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        # print(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        # print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        # print("Exactitud:"    , accuracy_score(y_test, yhat))
        # print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        # print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        # print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        # print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO NORMALIZADO SIN STRATIFY EN Y')
        return f1_score(y_test, yhat, average="macro")
    elif(stratify)and not(normalizar):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify=y)
        model = RandomForestClassifier(max_depth=i)
        model.fit(X_train, y_train) 
        yhat = model.predict(X_test)
        # print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        # print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
        # print("Exactitud:"    , accuracy_score(y_test, yhat))
        # print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
        # print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
        # print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
        # print("ROC AUC:", roc_auc_score(y_test, yhat))
        # confusion_matrix(y_test, yhat, labels = [0, 1])
        # plt.figure(figsize=(10,10))
        # sns.heatmap(confusion_matrix(y_test, yhat, labels = [0, 1]), annot=True, cmap='rainbow')
        # plt.xlabel('Predicción')
        # plt.ylabel('Verdad');
        # plt.title(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
        return f1_score(y_test, yhat, average="macro")
    else:
         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
         model = RandomForestClassifier(max_depth=i)
         model.fit(X_train, y_train) 
         yhat = model.predict(X_test)
         # print(f'MODELO SIN NORMALIZADO CON STRATIFY EN Y')
         # print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
         # print("Exactitud:"    , accuracy_score(y_test, yhat))
         # print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
         # print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
         # print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
         # print("ROC AUC:", roc_auc_score(y_test, yhat))
         return f1_score(y_test, yhat, average="macro")
    
    